# Audio Tagging (Classification)

In [10]:
#importing necessary packages
import edward as ed
import numpy as np
import pandas as pd
import tensorflow as tf

**Goal:** Develop an automatic, general-purpose audio tagging system capable of accurately classifying sound collections for a wide range of real-world environments.

## Data

The original dataset is taken from Kaggle (Audio Tagging competition). The samples (~20,000 WAV files) are generated from Freesound's library and include things like musical instruments, domestic sounds, and animals. Each input represents a WAV file with a corresponding annotative label. The dataset also includes a boolean column indicating whether the label was manually verified by a human.

In [11]:
train = pd.read_csv("data_overview/train.csv")
train.head()

,fname,label,manually_verified
0,00044347.wav,Hi-hat,0
1,001ca53d.wav,Saxophone,1
2,002d256b.wav,Trumpet,0
3,0033e230.wav,Glockenspiel,1
4,00353774.wav,Cello,1


The train set includes ~9500 audio files associated with 41 unique labels, each generated by Google’s AudioSet ontology.

In [13]:
print('Number of training examples (audiofiles): {}'.format(train.shape[0]))
print('Number of unique categories: {}'.format(train['label'].nunique()))

Number of training examples (audiofiles): 9473
Number of unique categories: 41


## Model

## Inference

## Critisism